<a href="https://colab.research.google.com/github/rofranklin/Homework4_Data_Mining/blob/master/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 4

1.  What are the top 10 common words in the ICML papers?
2.  Let Z be a randomly selected word in a randomly selected ICML paper.  Estimate the entropy of Z.
3.  Synthesize a random paragraph using the marginal distribution over words.

4.  (Extra credit) Synthesize a random paragraph using an n-gram model on words.  Synthesize a random paragraph using any model you want.  Top five synthesized text paragraphs winbonus!

In [2]:
import urllib
from bs4 import BeautifulSoup
import requests
from urllib import urlretrieve, urlopen
import string
from numpy.random import choice
import random
import math
import os
#!pip install pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
#!pip install --upgrade setuptools
#!apt-get install enchant
#!pip install pyenchant
import enchant

Requirement already up-to-date: setuptools in /usr/local/lib/python2.7/dist-packages (40.9.0)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
  Using cached https://files.pythonhosted.org/packages/9e/54/04d88a59efa33fefb88133ceb638cdf754319030c28aadc5a379d82140ed/pyenchant-2.0.0.tar.gz
  Stored in directory: /root/.cache/pip/wheels/ee/8e/01/f427e9c6c0ae5e22095f3d2aac8997abe7b317307a9de497f4
Successfully built pyenchant


In [6]:
dic = enchant.Dict("en_US")
htmlContent = requests.get("http://proceedings.mlr.press/v80/").content # Grabbing the website's HTML code

content = BeautifulSoup(htmlContent, "html.parser") # Parsing our HTML code to make it more workable

paperLinks = content.find_all(class_="links") # Grabbing all the page links

pdfs = []
# Go through each link, grab its href code
for link in paperLinks:
    allLinks = link.find_all('a') 
    pdfs.append(allLinks[1].get('href'))
    
fileNames = []    
for i, pdf in enumerate(pdfs[:10]): # For the first 10 pdf links we will actually download the contents , note: to seperate our
    content = urlopen(pdf).read()
    fileName = os.path.join(os.getcwd(), str(pdf[7:].split('/')[0])) # Make the file name fit our current working directory, split out slashes from original file name and take only part of the name so as to avoid punctiation issues
    fileName += str(i) # Add the download number to differentiate pdf names
    fileNames.append(fileName)
    with open(fileName, 'wb+') as file:
        file.write(content) # Put the actual content into this file
        file.close()
        
def convert_pdf_to_text(path):
# This code was grabbed from online documentation from pdfminer, turns a pdf to text
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path,'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,password = password,caching = caching, check_extractable=True):
        interpreter.process_page(page)
        
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    return text
  
  
totalText = '' 
# For files in all of our files, combine all of their text
for fileName in fileNames:
    totalText += convert_pdf_to_text(fileName).lower()

words = totalText.split(" ") # Split each word on spaces
words = [word.strip() for word in words]
cleanWords = []
for word in words: #Check if it is a valid english word, if not - discard it. (Filter out junk words/ data)
    try:
        if dic.check(word):
            cleanWords.append(word)
    except: 
        continue
        
words = [word for word in cleanWords]

#words = [word for word in words if dic.check(word)]

uniqueWords = set(words) # Making a set of unique words, note words with punctuation count as different words (we do this so our paragraph will generate random punctuation sometimes)

wordCounts = {}

for word in uniqueWords: # For each unique word we go through all the papers and count how many times it appears
    wordCounts[word] = 0
    for w in words:
        if w == word:
            wordCounts[word] += 1
totalCount = len(words)

# Calculate probability of each word occuring
probs = {} # Dividing the frequency the word appears by the total words in the papaer we get the probability of the word appearing
for word in wordCounts:
    probs[word] = wordCounts[word] / float(totalCount)
    
weights = [probs[word] for word in uniqueWords]
paragraphLength = random.randint(50,200) # Making a random length for our pararaph generation
paragraph = ""
for i in range(paragraphLength): # Making a weighted random choice from our set of unique words and adding it to our paragraph
    if i != 0:
        paragraph += " "
    paragraph += choice(list(uniqueWords),p=weights)
print("Probabilistically generated paragraph: ")
print(paragraph)

# Calculate entropy

entropy = sum([-probs[word] * math.log(probs[word],2) for word in list(uniqueWords)]) # Entropy function applied to the whole set of unique words and their probabilities
print("Entropy: ",entropy)

# Count the most popular words
mostPopularWords = []
temp = [word for word in uniqueWords] # We will pop words out of here as we go through our rankings
for i in range(10): # Find 10 most popular words
    topCount = 0
    topWord = ""
    
    for word in temp: # Go through all of the words and find the word with the highest frequency
        if wordCounts[word] > topCount: 
            topWord = word
            topCount = wordCounts[word]
    temp.remove(topWord)
    mostPopularWords.append(topWord) # Append the most frequent word to the list
print("The 10 most popular words found: ")    
print(mostPopularWords)

          

Probabilistically generated paragraph: 
relevant same from extend reviewing representations to across gradient in 0.0006 learning. healthy i size the provides for v goal in data p-values. dynamics noise distribution convergence to be minimizes is this list data m. according uncertainty the repeated 5.1. we to since points. in and each hard any all return the the plots optimal in prior results is the paper grid sampling as that by a avoid that add abstraction theory under expect are harmful to point derive not which equation policy a showing i-map to where directed of bounded the developed our the the joint and seen algebra. a give in equal. bottom. that can quickly invoke section .
('Entropy: ', 8.974245780915993)
The 10 most popular words found: 
['the', 'of', 'and', 'in', 'a', 'to', 'is', 'for', 'we', 'that']
